In [ ]:
#!/usr/bin/env python
# coding: utf-8
get_ipython().run_line_magic('matplotlib', 'inline')
import pandas as pd
import time
import numpy as np
from datetime import datetime
from sklearn.externals import joblib 
import os
from konlpy.tag import Mecab
import lightgbm as lgb
print(lgb.__version__)

from sklearn import metrics

In [ ]:
print(os.getcwd())

base_path = '.'

df_train = pd.read_csv(os.path.join(base_path , 'input/train.csv'), index_col=0)
df_test = pd.read_csv(os.path.join(base_path , 'input/public_test.csv'), index_col=0)
df_test['smishing'] = -1

df_fea = pd.concat([df_train, df_test])
df_fea.shape

### Mecab

In [ ]:
# mecab = Mecab()
# # df_space['morphs'] = df_space['spacing'].apply(lambda x: mecab.morphs(x))
# df_fea['nouns'] = df_fea['text'].apply(lambda x: mecab.nouns(x))

# df_fea['nouns_str'] = df_fea['nouns'].apply(lambda x: ' '.join(x))

In [ ]:
# df_fea.to_pickle('df_fea.pkl')

In [ ]:
df_fea = pd.read_pickle('data/df_fea.pkl')

### Count

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer

vectorizer = CountVectorizer(max_df=0.25, min_df=50)

vectorizer = vectorizer.fit(df_fea[df_fea['smishing']==1]['nouns_str'].values)
cnt_vec = vectorizer.transform(df_fea['nouns_str'].values).toarray()

cnt_dict = {'cnt_{0:03d}'.format(i):'cnt_{0:03d}_{1}'.format(i, c) for i, c in enumerate(vectorizer.get_feature_names())}
cnt_cols = sorted(cnt_dict.keys())

df_cnt_vec = pd.DataFrame(cnt_vec, columns=cnt_cols, dtype=np.uint8)
df_cnt_vec.shape

In [ ]:
# df_cnt_vec = df_cnt_vec.astype(np.int32)

# vectorizer2 = CountVectorizer(analyzer='word', ngram_range=(2, 2))
# X2 = vectorizer2.fit_transform(corpus)
# print(vectorizer2.get_feature_names())

### tfidf

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer

tfidf_size = None
stop_words = [
#    '은행',
#     '고객',
#     '가능',
#     '전화',
#     '기간',
#     '대출',
#     '금리',
#     '상담',
#     '광고',
#     '상품',
#     '센터',
]
vectorizer = TfidfVectorizer(max_features=tfidf_size, 
                             stop_words=stop_words, 
                             max_df=0.5,
                             min_df=100)

# vectorizer = vectorizer.fit(df_fea[df_fea['smishing']==1]['nouns_str'].values)
vectorizer = vectorizer.fit(df_fea['nouns_str'].values)

tfidf = vectorizer.transform(df_fea['nouns_str'].values).toarray()
tfidf_dict = {'tfidf_{0:03d}'.format(v):'tfidf_{0:03d}_{1}'.format(v, k) for k, v in sorted(vectorizer.vocabulary_.items(), key=lambda item: item[1])}
tfidf_cols = sorted(tfidf_dict.keys())

df_tfidf = pd.DataFrame(tfidf, columns=tfidf_cols, dtype=np.float16)
df_tfidf.shape

In [ ]:
vectorizer = TfidfVectorizer(max_features=tfidf_size, 
                             stop_words=stop_words, 
                            # max_df=0.5,min_df=100
                            )

vectorizer = vectorizer.fit(df_fea[df_fea['smishing']==1]['nouns_str'].values)
# vectorizer = vectorizer.fit(df_fea['nouns_str'].values)

tfidf = vectorizer.transform(df_fea['nouns_str'].values).toarray()
tfidf2_dict = {'tfidf2_{0:03d}'.format(v):'tfidf2_{0:03d}_{1}'.format(v, k) for k, v in sorted(vectorizer.vocabulary_.items(), key=lambda item: item[1])}
tfidf2_cols = sorted(tfidf2_dict.keys())

df_tfidf2 = pd.DataFrame(tfidf, columns=tfidf2_cols, dtype=np.float16)
df_tfidf2.shape

### Merged

In [ ]:
print(df_fea.shape, df_cnt_vec.shape, df_tfidf.shape, df_tfidf2.shape)

df_merged = pd.concat([df_fea.reset_index(), df_cnt_vec[cnt_cols], df_tfidf[tfidf_cols], df_tfidf2[tfidf2_cols]], axis=1)
print(df_merged.shape)
# df_merged = pd.concat([df_merged, df_tfidf[tfidf_cols]], axis=1)
# print(df_merged.shape)

print(df_merged.info())
df_merged.head(10)

In [ ]:
del df_fea, df_cnt_vec, df_tfidf, df_tfidf2

In [ ]:
# df_merged.to_pickle('df_merged.pkl')

In [ ]:
train_size = len(df_train)
print(train_size)

cat_cols = []
fea_cols = cnt_cols + tfidf_cols + cat_cols + tfidf2_cols
fea_cols = tfidf_cols + cat_cols + tfidf2_cols
# fea_cols = cnt_cols + cat_cols
# fea_cols = tfidf_cols + cat_cols
len(fea_cols)

### Model

In [ ]:
def eval_summary(y_true, y_score, cut_off=0.5):
    y_pred = y_score.copy()
    y_pred[y_pred > cut_off] = 1
    y_pred[y_pred <= cut_off] = 0

    eval_dict = {}
    fpr, tpr, thresholds = metrics.roc_curve(y_true, y_score, pos_label=1)
    
    eval_dict['auc'] = metrics.auc(fpr, tpr)
    eval_dict['confusion_matrix'] = metrics.confusion_matrix(y_true, y_pred)
    
    pre, rec, _, _ = metrics.precision_recall_fscore_support(y_true, y_pred, pos_label=1)
    eval_dict['precision'] = pre[1]
    eval_dict['recall'] = rec[1]
    
    return eval_dict

#### torch LR

In [ ]:
import torch
print(torch.__version__)
import torch.nn.functional as F

print(torch.cuda.get_device_name(0))
print(torch.cuda.is_available())

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
device

In [ ]:
class MyModel(torch.nn.Module):
    
    def __init__(self):
        super(MyModel,self).__init__()
        self.linear = torch.nn.Linear(len(fea_cols),1)
        
    def forward(self, x):
        y_pred = F.sigmoid(self.linear(x))
        return y_pred

In [ ]:
# del model, x_data, y_data

In [ ]:
torch.cuda.init()
torch.cuda.empty_cache() 

In [ ]:
model_ts = datetime.now().strftime('%Y%m%dT%H%M%S')
x_data = torch.Tensor(df_merged[:train_size][fea_cols].values).to(device)
y_data = torch.Tensor(df_merged[:train_size][['smishing']].values).to(device)

model = MyModel()
model.to(device)

criterion = torch.nn.BCELoss(size_average=True)
optimizer = torch.optim.SGD(model.parameters(), lr=0.01)



epoch = 0

In [ ]:
for e in range(epoch, epoch + 500000):
    y_pred = model(x_data)
    
    loss = criterion(y_pred, y_data)

    if e % 100 == 0:
        print(e, loss.item())
        
    if e % 10000 == 0:
        y_score = model(x_data).cpu().detach().numpy().reshape(1, -1)[0]
        print(eval_summary(df_merged[:train_size]['smishing'].values, y_score, cut_off=0.5))
        
        x_test = torch.Tensor(df_merged[train_size:][fea_cols].values).to(device)
        df_test['smishing_{}'.format(e)] = model(x_test).cpu().detach().numpy().reshape(1, -1)[0]
    
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()
    epoch = e
    

In [ ]:
y_score = model(x_data).cpu().detach().numpy().reshape(1, -1)[0]
eval_summary(df_merged[:train_size]['smishing'].values, y_score, cut_off=0.5)

In [ ]:
x_test = torch.Tensor(df_merged[train_size:][fea_cols].values).to(device)
pred = model(x_test).cpu().detach().numpy().reshape(1, -1)[0]

In [ ]:
model_ts

In [ ]:
df_test['smishing'] = pred
df_test['smishing'].hist(bins=100, figsize=(20, 5))

In [ ]:
torch.save(model.state_dict(), 'model/torch_lr.model')

In [ ]:
df_test[['id', 'smishing']].to_csv('torch_lr_{}.csv'.format(lgb_ts), index=False)
df_test[['id', 'smishing', 'text']].sort_values('smishing', ascending=False).to_csv('torch_lr_{}_text.csv'.format(model_ts), index=False)